<a href="https://colab.research.google.com/github/jeyasubha-26/mindmapGenerator/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gradio

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr
import torch

# Load FLAN-T5 model
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Function to summarize if needed
def summarize_if_needed(text, complexity):
    if len(text.split()) > 120 and complexity < 7:  # more than ~2 paras
        prompt = f"Summarize the following text clearly:\n\n{text}"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
        outputs = model.generate(**inputs, max_new_tokens=150)
        summarized = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return summarized
    return text

# Function to create mindmap
def generate_mindmap(text, complexity):
    # Optional summarization
    processed_text = summarize_if_needed(text, complexity)

    # Build prompt
    prompt = (
        f"Create a tree-structured mindmap from the text below. "
        f"Use clear hierarchy and structure. Complexity level: {complexity}/10.\n\n"
        f"Text:\n{processed_text}"
    )

    # Generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=250)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Try formatting output like a tree
    lines = result.split('. ')
    tree = "🧠 Mindmap\n"
    for i, line in enumerate(lines):
        if line.strip():
            indent = "  " * (i % (complexity//2 + 1))
            tree += f"{indent}├─ {line.strip().rstrip('.')}\n"

    return tree

# Gradio UI
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown("## 💡 Enter Paragraph(s)")
    input_text = gr.Textbox(lines=6, placeholder="Enter your paragraph(s) here...", label="Enter Paragraph(s)")

    complexity_slider = gr.Slider(1, 10, value=5, label="Mindmap Complexity (1 = Short, 10 = Detailed)", interactive=True)
    output_text = gr.Textbox(label="Generated Mindmap", lines=15)

    btn = gr.Button("🧠 Generate Mindmap")
    btn.click(generate_mindmap, inputs=[input_text, complexity_slider], outputs=output_text)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d902595cd08ab346e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
